# Optima Data Report Cases | 6511

In [1]:
import pandas as pd
from collections import Counter

In [2]:
def extract_values(df, column, values):
    d = df[df[column].isin(values)]
    print(len(set(d.index)))
    return set(d.index), d 

In [3]:
def scan_unique(df, column):
    return df[column].unique()

In [4]:
def col_name(name):
    name = name.lower()
    return data.columns[[name in i.lower() for i in data.columns]]

In [5]:
def get_missing_datasets(column):
    print("null:", data[data[column].isna()].shape)
    print("not asked:", data[data[column].isin([9])].shape)
    print("not known:", data[data[column].isin([8])].shape)
    return data[data[column].isna()], data[data[column].isin([8])], pd.concat([data[data[column].isna()], data[data[column].isin([8, 9])]])

In [6]:
current_treatment = ["CURRENT MEDICATION: TREATMENT "+str(i) for i in range(1, 21)]

In [7]:
data = pd.read_excel("Data Request Jan 2019 (Optima, Lead, Challenge)_final (1).xlsx")

In [8]:
data = data.drop_duplicates(['GLOBAL_PATIENT_DB_ID','EPISODE_DATE'])

In [9]:
minimental = "CAMDEX SCORES: MINI MENTAL SCORE"

In [10]:
data = data[data[minimental].between(0, 30, inclusive=True)]

In [11]:
data.shape

(6511, 1593)

In [12]:
data.shape

(6511, 1593)

## Petersen_MCI

In [13]:
petersen_mci, petersen_mci_type = col_name("petersen")

In [14]:
petersen_mci, petersen_mci_type

('OPTIMA DIAGNOSES V 2010: PETERSEN MCI',
 'OPTIMA DIAGNOSES V 2010: PETERSEN MCI TYPE')

In [15]:
cdr_score = col_name("cdr")[1]

In [16]:
cdr_score

'CLINICAL DEMENTIA RATING: OVERALL CDR SCORE'

In [17]:
_, _, petersen_data = get_missing_datasets(petersen_mci)

null: (22, 1593)
not asked: (3859, 1593)
not known: (4, 1593)


In [18]:
petersen_data.shape

(3885, 1593)

### Petersen | Extraction

In [19]:
# scan_unique(petersen_data, cdr_score) #cdr score not in range

In [20]:
# scan_unique(petersen_data, petersen_mci_type)

In [21]:
p_0, _ = extract_values(petersen_data, cdr_score, [1, 2])

19


In [22]:
p_1, _ = extract_values(petersen_data, petersen_mci_type, [1, 2, 3, 4])

0


In [23]:
petersen_update = p_0 | p_1

In [24]:
no_petersen_update, _ = extract_values(petersen_data, cdr_score, [0])

8


In [25]:
len(petersen_update), len(no_petersen_update)

(19, 8)

In [26]:
Counter(data.loc[petersen_update | no_petersen_update, petersen_mci])

Counter({9.0: 26, nan: 1})

In [27]:
data.loc[no_petersen_update, petersen_mci] = 0

In [28]:
data.loc[petersen_update, petersen_mci] = 1

In [29]:
Counter(data.loc[petersen_update | no_petersen_update, petersen_mci])

Counter({1.0: 19, 0.0: 8})

In [30]:
len(petersen_update | no_petersen_update)

27

## DEPRESSIVE_ILLNESS

#### To be confirmed:
- "PRESENT STATE 13-46: (038) REASON FOR DEPRESSION" Values to consider
- "PRESENT STATE 13-46: (037) DURATION OF DEPRESSION" Values to Consider
- "CAMDEX SCORES: DEPRESSION SCALE" Numeric Value
- "CAMDEX SCORES: GERIATRIC DEPRESSION SCORE" Numeric Value
- "NPI: DEPRESSION/DYSPHORIA: F X S" Numeric Value

In [31]:
col_name("depress")

Index(['CAMDEX ADMINISTRATION 1-12: EST OF SEVERITY OF DEPRESSION',
       'PRESENT STATE 13-46: (036) FEELING DEPRESSED',
       'PRESENT STATE 13-46: (037) DURATION OF DEPRESSION',
       'PRESENT STATE 13-46: (038) REASON FOR DEPRESSION',
       'PRESENT STATE 13-46: (038) REASON FOR DEPRESSION: TEXT',
       'INTERVIEWER OBS 188-212: (196) DEPRESSED MOOD',
       'DIAGNOSIS 334-351: DEPRESSIVE ILLNESS',
       'DIAGNOSIS 334-351: SEVERITY OF DEPRESSION',
       'CAMDEX SCORES: DEPRESSION SCALE',
       'CAMDEX SCORES: GERIATRIC DEPRESSION SCORE',
       'MEDICAL ASSESSMENT V 2010: DEPRESSION',
       'MEDICAL ASSESSMENT V 2010: DEPRESSION TREATED BY DOCTOR',
       'HACHINSKI ISCHAEMIC: DEPRESSIVE SYMPTOMATOLOGY',
       'NPI: DEPRESSION/DYSPHORIA: FREQUENCY',
       'NPI: DEPRESSION/DYSPHORIA: SEVERITY',
       'NPI: DEPRESSION/DYSPHORIA: F X S',
       'NPI: DEPRESSION/DYSPHORIA: DISTRESS'],
      dtype='object')

In [32]:
depressive_illness = 'DIAGNOSIS 334-351: DEPRESSIVE ILLNESS'

In [33]:
depression_columns = [['CAMDEX ADMINISTRATION 1-12: EST OF SEVERITY OF DEPRESSION', [1, 2, 3, 4]],
                      ['PRESENT STATE 13-46: (036) FEELING DEPRESSED', [1, 2]],
                      ['INTERVIEWER OBS 188-212: (196) DEPRESSED MOOD', [1]],
                      ['DIAGNOSIS 334-351: SEVERITY OF DEPRESSION', [1, 2, 3, 4]],
                      ['MEDICAL ASSESSMENT V 2010: DEPRESSION', [1]],
                      ['MEDICAL ASSESSMENT V 2010: DEPRESSION TREATED BY DOCTOR', [1]],
                      ['HACHINSKI ISCHAEMIC: DEPRESSIVE SYMPTOMATOLOGY', [1]],
                      ['NPI: DEPRESSION/DYSPHORIA: FREQUENCY', [1, 2, 3, 4]],
                      ['NPI: DEPRESSION/DYSPHORIA: SEVERITY', [1, 2, 3]],
                      ['NPI: DEPRESSION/DYSPHORIA: DISTRESS', [1, 2, 3, 4, 5]],
                      ['SPECT SCAN: DIAGNOSTIC ASSESSMENT', [8]]]

In [34]:
for t in current_treatment:
    depression_columns.append([t, [80]])

In [35]:
no_depression_columns = [['CAMDEX ADMINISTRATION 1-12: EST OF SEVERITY OF DEPRESSION', [0]],
                      ['PRESENT STATE 13-46: (036) FEELING DEPRESSED', [0]],
                      ['INTERVIEWER OBS 188-212: (196) DEPRESSED MOOD', [0]],
                      ['DIAGNOSIS 334-351: SEVERITY OF DEPRESSION', [0]],
                      ['MEDICAL ASSESSMENT V 2010: DEPRESSION', [0]],
                      ['MEDICAL ASSESSMENT V 2010: DEPRESSION TREATED BY DOCTOR', [0]],
                      ['HACHINSKI ISCHAEMIC: DEPRESSIVE SYMPTOMATOLOGY', [0]],
                      ['NPI: DEPRESSION/DYSPHORIA: FREQUENCY', [0]],
                      ['NPI: DEPRESSION/DYSPHORIA: SEVERITY', [0]],
                      ['NPI: DEPRESSION/DYSPHORIA: DISTRESS', [0]]]

In [36]:
_, _, depressive_illness_data = get_missing_datasets(depressive_illness)

null: (3079, 1593)
not asked: (3380, 1593)
not known: (0, 1593)


In [37]:
depressive_illness_data.shape

(6459, 1593)

### Depressive Illness | Extraction

In [38]:
dil_update = set()

In [39]:
for idx, a in enumerate(depression_columns):
    print("column:", a[0])
    dil_update |= extract_values(depressive_illness_data, a[0], a[1])[0]

column: CAMDEX ADMINISTRATION 1-12: EST OF SEVERITY OF DEPRESSION
1109
column: PRESENT STATE 13-46: (036) FEELING DEPRESSED
1189
column: INTERVIEWER OBS 188-212: (196) DEPRESSED MOOD
403
column: DIAGNOSIS 334-351: SEVERITY OF DEPRESSION
553
column: MEDICAL ASSESSMENT V 2010: DEPRESSION
72
column: MEDICAL ASSESSMENT V 2010: DEPRESSION TREATED BY DOCTOR
71
column: HACHINSKI ISCHAEMIC: DEPRESSIVE SYMPTOMATOLOGY
33
column: NPI: DEPRESSION/DYSPHORIA: FREQUENCY
364
column: NPI: DEPRESSION/DYSPHORIA: SEVERITY
317
column: NPI: DEPRESSION/DYSPHORIA: DISTRESS
312
column: SPECT SCAN: DIAGNOSTIC ASSESSMENT
7
column: CURRENT MEDICATION: TREATMENT 1
265
column: CURRENT MEDICATION: TREATMENT 2
135
column: CURRENT MEDICATION: TREATMENT 3
84
column: CURRENT MEDICATION: TREATMENT 4
56
column: CURRENT MEDICATION: TREATMENT 5
40
column: CURRENT MEDICATION: TREATMENT 6
17
column: CURRENT MEDICATION: TREATMENT 7
11
column: CURRENT MEDICATION: TREATMENT 8
9
column: CURRENT MEDICATION: TREATMENT 9
2
column: C

In [40]:
no_dil_update = set()

In [41]:
for idx, a in enumerate(no_depression_columns):
    print("column:", a[0])
    no_dil_update |= extract_values(depressive_illness_data, a[0], a[1])[0]

column: CAMDEX ADMINISTRATION 1-12: EST OF SEVERITY OF DEPRESSION
2889
column: PRESENT STATE 13-46: (036) FEELING DEPRESSED
2235
column: INTERVIEWER OBS 188-212: (196) DEPRESSED MOOD
3843
column: DIAGNOSIS 334-351: SEVERITY OF DEPRESSION
1151
column: MEDICAL ASSESSMENT V 2010: DEPRESSION
275
column: MEDICAL ASSESSMENT V 2010: DEPRESSION TREATED BY DOCTOR
307
column: HACHINSKI ISCHAEMIC: DEPRESSIVE SYMPTOMATOLOGY
134
column: NPI: DEPRESSION/DYSPHORIA: FREQUENCY
705
column: NPI: DEPRESSION/DYSPHORIA: SEVERITY
705
column: NPI: DEPRESSION/DYSPHORIA: DISTRESS
625


In [42]:
len(dil_update), len(no_dil_update)

(2546, 4891)

In [43]:
Counter(data.loc[dil_update | no_dil_update, depressive_illness])

Counter({9: 3363, nan: 2080})

In [44]:
data.loc[no_dil_update, depressive_illness] = 0

In [45]:
data.loc[dil_update, depressive_illness] = 1

In [46]:
Counter(data.loc[dil_update | no_dil_update, depressive_illness])

Counter({0: 2897, 1: 2546})

In [47]:
len(dil_update | no_dil_update)

5443

## CERBRO-VASCULAR_DISEASE_PRESENT

In [48]:
cerbro_vascular = col_name("cerbro")[0]

In [49]:
cerbro_vascular

'OPTIMA DIAGNOSES V 2010: CERBRO-VASCULAR DISEASE PRESENT'

In [50]:
col_name("strok")

Index(['HISTORY PATIENT 74-119: (076) HISTORY OF STROKE',
       'HISTORY PATIENT 74-119: (099) FAMILY HISTORY OF STROKE',
       'HISTORY PATIENT 74-119: (099) FAMILY HISTORY OF STROKE: FEMALES',
       'HISTORY PATIENT 74-119: (099) FAMILY HISTORY OF STROKE: MALES',
       'HISTORY PATIENT 74-119: (099) FAMILY HISTORY OF STROKE: TEXT',
       'MEDICAL ASSESSMENT V 2010: STROKE',
       'HACHINSKI ISCHAEMIC: HISTORY OF STROKE'],
      dtype='object')

In [51]:
stroke_columns = ['HISTORY PATIENT 74-119: (076) HISTORY OF STROKE',
                  'MEDICAL ASSESSMENT V 2010: STROKE',
                  'HACHINSKI ISCHAEMIC: HISTORY OF STROKE']

In [52]:
_, _, cerbro_vascular_data = get_missing_datasets(cerbro_vascular)

null: (22, 1593)
not asked: (4164, 1593)
not known: (393, 1593)


In [53]:
cerbro_vascular_data.shape

(4579, 1593)

### Cerbro Vascular | Extraction

In [54]:
str_1, _ = extract_values(cerbro_vascular_data, stroke_columns[0], [1, 2, 3])

255


In [55]:
str_2, _ = extract_values(cerbro_vascular_data, stroke_columns[1], [1])

0


In [56]:
str_3, _ = extract_values(cerbro_vascular_data, stroke_columns[2], [1])

1


In [57]:
no_str_1, _ = extract_values(cerbro_vascular_data, stroke_columns[0], [0])

2729


In [58]:
no_str_2, _ = extract_values(cerbro_vascular_data, stroke_columns[1], [0])

10


In [59]:
no_str_3, _ = extract_values(cerbro_vascular_data, stroke_columns[2], [0])

4


In [60]:
str_update = str_1 | str_2 | str_3

In [61]:
no_str_update = no_str_1 | no_str_2 | no_str_3

In [62]:
Counter(data.loc[str_update | no_str_update, cerbro_vascular])

Counter({9.0: 2760,
         8.0: 224,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1})

In [63]:
data.loc[no_str_update, cerbro_vascular] = 0

In [64]:
data.loc[str_update, cerbro_vascular] = 1

In [65]:
len(str_update), len(no_str_update), len(str_update | no_str_update)

(256, 2736, 2991)

In [66]:
Counter(data.loc[str_update | no_str_update, cerbro_vascular])

Counter({0.0: 2735, 1.0: 256})

## ANXIETY/PHOBIC

In [67]:
col_name("anxi")

Index(['PRESENT STATE 47-73: (049) ANXIOUS',
       'PRESENT STATE 47-73: (051) ANXIOUS SITUATIONS',
       'PRESENT STATE 47-73: (051) ANXIOUS SITUATIONS: TEXT',
       'INTERVIEWER OBS 188-212: (195) ANXIOUS OR FEARFUL',
       'DIAGNOSIS 334-351: ANXIETY/PHOBIC',
       'MEDICAL ASSESSMENT V 2010: ANXIETY', 'NPI: ANXIETY: FREQUENCY',
       'NPI: ANXIETY: SEVERITY', 'NPI: ANXIETY: F X S',
       'NPI: ANXIETY: DISTRESS'],
      dtype='object')

In [68]:
anxiety_phobic = "DIAGNOSIS 334-351: ANXIETY/PHOBIC"

In [69]:
anxiety_columns = [['PRESENT STATE 47-73: (049) ANXIOUS', [1]],
                   ['PRESENT STATE 47-73: (051) ANXIOUS SITUATIONS', [1]],
                   ['INTERVIEWER OBS 188-212: (195) ANXIOUS OR FEARFUL', [1]],
                   ['MEDICAL ASSESSMENT V 2010: ANXIETY', [1]],
                   ['NPI: ANXIETY: FREQUENCY', [1, 2, 3, 4]],
                   ['NPI: ANXIETY: SEVERITY', [1, 2, 3]],
                   ['NPI: ANXIETY: F X S', [2.,  8.,  3.,  1.,  4.,  6., 12.,  9.,  5.]],
                   ['NPI: ANXIETY: DISTRESS', [1, 2, 3, 4, 5]]]

In [70]:
no_anxiety_columns = [['PRESENT STATE 47-73: (049) ANXIOUS', [0]],
                   ['PRESENT STATE 47-73: (051) ANXIOUS SITUATIONS', [0]],
                   ['INTERVIEWER OBS 188-212: (195) ANXIOUS OR FEARFUL', [0]],
                   ['MEDICAL ASSESSMENT V 2010: ANXIETY', [0]],
                   ['NPI: ANXIETY: FREQUENCY', [0]],
                   ['NPI: ANXIETY: SEVERITY', [0]],
                   ['NPI: ANXIETY: F X S', [0]],
                   ['NPI: ANXIETY: DISTRESS', [0]]]

In [71]:
for t in current_treatment:
    anxiety_columns.append([t, [85]])

In [72]:
_, _, anxiety_phobic_data = get_missing_datasets(anxiety_phobic)

null: (3088, 1593)
not asked: (3408, 1593)
not known: (0, 1593)


In [73]:
anxiety_phobic_data.shape

(6496, 1593)

In [74]:
# anxiety_phobic_null[anxiety_columns[7][0]].unique() # should be modified above

In [75]:
# anxiety_phobic_9[anxiety_columns[7][0]].unique() # should be modified above

In [76]:
# for i, _ in anxiety_columns:
#     print(i)
#     print(scan_unique(data, i))

### Anxiety Phobic | Extraction

In [77]:
anx_update = set()

In [78]:
for idx, a in enumerate(anxiety_columns):
    print("column:", a[0])
    anx_update |= extract_values(anxiety_phobic_data, a[0], a[1])[0]

column: PRESENT STATE 47-73: (049) ANXIOUS
748
column: PRESENT STATE 47-73: (051) ANXIOUS SITUATIONS
694
column: INTERVIEWER OBS 188-212: (195) ANXIOUS OR FEARFUL
608
column: MEDICAL ASSESSMENT V 2010: ANXIETY
82
column: NPI: ANXIETY: FREQUENCY
268
column: NPI: ANXIETY: SEVERITY
225
column: NPI: ANXIETY: F X S
360
column: NPI: ANXIETY: DISTRESS
161
column: CURRENT MEDICATION: TREATMENT 1
43
column: CURRENT MEDICATION: TREATMENT 2
29
column: CURRENT MEDICATION: TREATMENT 3
23
column: CURRENT MEDICATION: TREATMENT 4
10
column: CURRENT MEDICATION: TREATMENT 5
12
column: CURRENT MEDICATION: TREATMENT 6
4
column: CURRENT MEDICATION: TREATMENT 7
1
column: CURRENT MEDICATION: TREATMENT 8
1
column: CURRENT MEDICATION: TREATMENT 9
2
column: CURRENT MEDICATION: TREATMENT 10
1
column: CURRENT MEDICATION: TREATMENT 11
0
column: CURRENT MEDICATION: TREATMENT 12
0
column: CURRENT MEDICATION: TREATMENT 13
0
column: CURRENT MEDICATION: TREATMENT 14
0
column: CURRENT MEDICATION: TREATMENT 15
0
column: 

In [79]:
no_anx_update = set()

In [80]:
for idx, a in enumerate(no_anxiety_columns):
    print("column:", a[0])
    no_anx_update |= extract_values(anxiety_phobic_data, a[0], a[1])[0]

column: PRESENT STATE 47-73: (049) ANXIOUS
2672
column: PRESENT STATE 47-73: (051) ANXIOUS SITUATIONS
2691
column: INTERVIEWER OBS 188-212: (195) ANXIOUS OR FEARFUL
3674
column: MEDICAL ASSESSMENT V 2010: ANXIETY
265
column: NPI: ANXIETY: FREQUENCY
796
column: NPI: ANXIETY: SEVERITY
796
column: NPI: ANXIETY: F X S
707
column: NPI: ANXIETY: DISTRESS
763


In [81]:
Counter(data.loc[anx_update | no_anx_update, anxiety_phobic])

Counter({9: 3367, nan: 1988})

In [82]:
data.loc[no_anx_update, anxiety_phobic] = 0

In [83]:
data.loc[anx_update, anxiety_phobic] = 1

In [84]:
Counter(data.loc[anx_update | no_anx_update, anxiety_phobic])

Counter({1: 1934, 0: 3421})

In [85]:
len(anx_update), len(no_anx_update), len(anx_update | no_anx_update)

(1934, 4896, 5355)

In [86]:
data.to_excel("Optima_Data_Report_Cases_6511_filled.xlsx")

In [87]:
data.to_pickle("Optima_Data_Report_Cases_6511_filled_pickle")

In [88]:
data

,GLOBAL_PATIENT_DB_ID,STUDY_ID,STUDY,DOD,Age At Episode,GENDER,EPISODE_DATE,EPISODE,CAMDEX ADMINISTRATION 1-12: DATE OF EXAMINATION,CAMDEX ADMINISTRATION 1-12: EST SEVERITY OF DEMENTIA,...,GENERAL INFORMATION: NSAIDS: DURATION,GENERAL INFORMATION: ASPIRIN,GENERAL INFORMATION: ASPIRIN: DURATION,GENERAL INFORMATION: OESTROGEN,GENERAL INFORMATION: OESTROGEN: DURATION,GENERAL INFORMATION: OESTROGEN: MENOPAUSE,GENERAL INFORMATION: ANTIHYPERTENSIVES,GENERAL INFORMATION: ANTIHYPERTENSIVES: DURATION,GENERAL INFORMATION: DIABETES,GENERAL INFORMATION: DIABETES: DURATION
0,1,C1048,Challenge,NaT,71,Male,1998-01-13,1,1998-01-13 00:00:00,0.0,...,1.0,2.0,NaN,9.0,NaN,NaN,0.0,36.0,0.0,NaN
2,1,C1048,Challenge,NaT,73,Male,2000-01-20,5,2000-01-20 00:00:00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,C1048,Challenge,NaT,75,Male,2002-02-21,9,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,C1048,Challenge,NaT,75,Male,2002-10-28,11,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,C1048,Challenge,NaT,77,Male,2004-06-22,13,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9590,8754,N0262,LEAD Cohort,NaT,69,Male,2012-10-25,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9591,8754,N0262,LEAD Cohort,NaT,70,Male,2014-03-18,5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9592,8754,N0262,LEAD Cohort,NaT,71,Male,2015-03-19,7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9593,8760,N0264,LEAD Cohort,NaT,73,Female,2011-12-05,1,2011-12-05 00:00:00,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
